## Testing DFA learning with a string rewriting system

In [1]:
import copy, sys, pandas as pd
import random 
import cProfile

sys.path.append("../")
sys.path.append("../inferring")
sys.path.append("../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.advice_systems.SRS as SRS

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS 

from inferring.InferringDFA import InferringDFA

In [2]:
def run_learning_process(test, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(test.dfa)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [3]:
class test_example():
    def __init__(self, dfa, descrip=''):
        self.dfa = copy.deepcopy(dfa)
        self.descrip = descrip

In [4]:
def create_table(columns, data):
    columns = pd.MultiIndex.from_tuples(columns)
    df = pd.DataFrame(data, columns=columns)

    avg_mq_red = df[(('Reduction', 'MQ'))].mean()
    avg_eq_red = df[(('Reduction', 'EQ'))].mean()

    max_mq_red = df[(('Reduction', 'MQ'))].max()
    max_eq_red = df[(('Reduction', 'EQ'))].max()

    min_mq_red = df[(('Reduction', 'MQ'))].min()
    min_eq_red = df[(('Reduction', 'EQ'))].min()

    empty_row = pd.DataFrame([[""] * df.shape[1]], columns=df.columns)

    def set_params(row, params):
        for k, v in params:
            row[columns[k]] = v 

    max_red_row = copy.deepcopy(empty_row)
    set_params(row=max_red_row, params=[(0, "Max reduction"), (-2, int(max_mq_red)), (-1, int(max_eq_red))])
    min_red_row = copy.deepcopy(empty_row)
    set_params(row=min_red_row, params=[(0, "Min reduction"), (-2, int(min_mq_red)), (-1, int(min_eq_red))])
    avg_red_row = copy.deepcopy(empty_row)
    set_params(row=avg_red_row, params=[(0, "Average reduction"), (-2, int(avg_mq_red)), (-1, int(avg_eq_red))])

    rows = [empty_row, max_red_row, min_red_row, avg_red_row]
    for row in rows:
        df = pd.concat([df, row], ignore_index=True)

    df[('Reduction', 'MQ')] = df[('Reduction', 'MQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)
    df[('Reduction', 'EQ')] = df[('Reduction', 'EQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)
    return df    

### Concurrent systems

#### Convolution of patterns-dfa (dfa recognizing patterns)

In [6]:
def random_patterns(input_signs, n=5, max_length=10):
    patterns = []
    for _ in range(n):
        p = ''.join(random.choices(population=input_signs,k=random.randint(max_length//2, max_length)))
        while p in set(patterns):
            p = ''.join(random.choices(population=input_signs,k=random.randint(max_length//2, max_length)))
        patterns.append(p)
    return patterns

In [7]:
class RecordPattern:
    def __init__(self, d1, d1_type, d2, d2_type, conv, cnt_normal, cnt_with_advice):
        pdfa_types = {DFA.OR_TYPE_PATTERN_DFA: "OR", DFA.AND_TYPE_PATTERN_DFA:"AND"}
        
        self.d1 = d1
        self.d2 = d2
        self.d1_type = pdfa_types[d1_type]
        self.d2_type = pdfa_types[d2_type] 
        
        self.conv = conv
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)

    def print_record(self):
        print(f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}, (mq, eq) = {self.cnt_normal} vs (mq, eq) = {self.cnt_with_advice}, mq_reduction = {self.mq_reduction}, eq_reduction ={self.eq_reduction}") 

In [16]:
seeds =  [i+1 for i in range(6)]
input_signs = ['a', 'b', 'c', 'd']

number_of_itreations = len(seeds)
number_of_patterns = 2
max_pattern_length = 8

results = []

i = 0 
while i < number_of_itreations:
    random.seed(seeds[i])
    i += 1 
    print(f"iter nr: {i}")

    dfa1, dfa2 = DFA(), DFA()
    dfa1.create_pattern_dfa(input_signs=input_signs, patterns=random_patterns(input_signs, number_of_patterns, max_pattern_length), _type=DFA.OR_TYPE_PATTERN_DFA if i&1 else DFA.AND_TYPE_PATTERN_DFA)
    dfa2.create_pattern_dfa(input_signs=input_signs, patterns=random_patterns(input_signs, number_of_patterns, max_pattern_length), _type=DFA.OR_TYPE_PATTERN_DFA if i&1 else DFA.AND_TYPE_PATTERN_DFA)

    d1, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa1)))
    d2, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa2)))

    conv_dfa = DFA() 
    conv_dfa.create_convolution(dfa1=copy.deepcopy(d1), dfa2=copy.deepcopy(d2))

    d, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=conv_dfa), advice_system=SRS(), check_consistency=True) 
    d.type = DFA.CONV_DFA
    print(f"rozmiar conwolucji: {d.Q}")
    _, cnt_normal, _ = run_learning_process(test=test_example(dfa=d), advice_system=None, check_consistency=False)

    results.append(RecordPattern(d1=d1.Q, d1_type = dfa1.type, d2=d2.Q, d2_type=dfa2.type, conv=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice)))

iter nr: 1
rozmiar conwolucji: 48
iter nr: 2
rozmiar conwolucji: 600
iter nr: 3
rozmiar conwolucji: 104
iter nr: 4
rozmiar conwolucji: 520
iter nr: 5
rozmiar conwolucji: 143
iter nr: 6
rozmiar conwolucji: 720


In [17]:
results.sort(key=lambda x: -100000 +x.conv if x.d1_type=="OR" else x.conv)

In [18]:
columns = [('Target language', 'conv(DFA1, DFA2)'), ('Target language', 'DFA1'), ('Target language', 'DFA2'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.conv for r in results],
    columns[1]: [(r.d1, r.d1_type) for r in results],
    columns[2]: [(r.d2, r.d2_type) for r in results],
    columns[3]: [r.cnt_normal[0] for r in results],
    columns[4]: [r.cnt_normal[1] for r in results],
    columns[5]: [r.cnt_with_advice[0][0] for r in results],
    columns[6]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[7]: [r.mq_reduction for r in results],
    columns[8]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

Target language                       L* without advice       \
    conv(DFA1, DFA2)       DFA1       DFA2                MQ   EQ   
0                 48    (6, OR)    (8, OR)             15213   28   
1                104   (13, OR)    (8, OR)             43115   34   
2                143   (13, OR)   (11, OR)             83309   43   
3                520  (26, AND)  (20, AND)           1529740  204   
4                600  (20, AND)  (30, AND)           4382243  218   
5                720  (24, AND)  (30, AND)           2606917  213   
6                                                                   
7      Max reduction                                                
8      Min reduction                                                
9  Average reduction                                                

  L* with advice           Reduction       
              MQ        EQ        MQ   EQ  
0           9752   (3, 28)       35%  89%  
1          27081   (4, 34)       37%  88%  
2          43707   (4, 40)       47%  90%  
3         436343  (4, 147)       71%  98%  
4         597198  (4, 183)       86%  98%  
5         800076  (4, 213)       69%  98%  
6                                          
7                                86%  98%  
8                                35%  88%  
9                                57%  93%

#### Convolution of 2 random DFA 

In [5]:
class Record:
    def __init__(self, d1, d2, conv, cnt_normal, cnt_with_advice):
        self.d1 = d1
        self.d2 = d2
        self.conv = conv
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
    def print_record(self):
        print(f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}, (mq, eq) = {self.cnt_normal} vs (mq, eq) = {self.cnt_with_advice}, mq_reduction = {self.mq_reduction}, eq_reduction ={self.eq_reduction}") 

In [ ]:
seeds =  [i+1 for i in range(10)]
input_signs = ['a', 'b', 'c', 'd']
max_number_of_states = 50
number_of_itreations = len(seeds)

results = []

i = 0 
while i < number_of_itreations: 
    random.seed(seeds[i])
    i += 1
    print(f"iter nr: {i}")

    dfa1 = DFA()
    dfa2 = DFA()

    dfa1.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    dfa2.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    
    d1, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa1)))   
    d2, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa2)))
    conv_dfa = DFA()
    conv_dfa.create_convolution(dfa1=d1, dfa2=d2) 
    
    d, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=conv_dfa), advice_system=SRS(), check_consistency=True) 
    d.type = DFA.CONV_DFA
    _, cnt_normal, _ = run_learning_process(test=test_example(dfa=d), advice_system=None, check_consistency=False)

    results.append(Record(d1=d1.Q, d2=d2.Q, conv=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice)))

In [161]:
results.sort(key=lambda x: x.conv)

In [ ]:
columns = [('Target language', 'conv(DFA1, DFA2)'), ('Target language', 'DFA1'), ('Target language', 'DFA2'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.conv for r in results],
    columns[1]: [r.d1 for r in results],
    columns[2]: [r.d2 for r in results],
    columns[3]: [r.cnt_normal[0] for r in results],
    columns[4]: [r.cnt_normal[1] for r in results],
    columns[5]: [r.cnt_with_advice[0][0] for r in results],
    columns[6]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[7]: [r.mq_reduction for r in results],
    columns[8]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

Target language           L* without advice      L* with advice     \
     conv(DFA1, DFA2) DFA1 DFA2                MQ   EQ             MQ EQ   
0                 675   25   27            506357   72         260570  1   
1                 945   35   27           3249401  250        1149944  3   
2                1088   32   34           1029383   93         442655  1   
3                1184   32   37           1949099  153         909933  2   
4                1184   32   37           3938459  262         915827  2   
5                1232   28   44           1381232  102         507200  1   
6                1890   42   45           4937053  240        1675933  1   
7                2150   50   43           3749849  119        1430162  2   
8                                                                          
9       Max reduction                                                      
10      Min reduction                                                      
11  Average reduction                                                      

   Reduction       
          MQ   EQ  
0        48%  98%  
1        64%  98%  
2        56%  98%  
3        53%  98%  
4        76%  99%  
5        63%  99%  
6        66%  99%  
7        61%  98%  
8                  
9        76%  99%  
10       48%  98%  
11       60%  98%

### Synchronizing words

In [114]:
class RecordSYNCH:
    def __init__(self, d, reset_word, cnt_normal, cnt_with_advice):
        self.d = d
        self.reset_word = reset_word
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
    def print_record(self):
        print(f"|d1| = {self.d}, |reset_word| = {len(self.reset_word)}, cnt_normal = {self.cnt_normal} vs cnt_with_advice = {self.cnt_with_advice}")

In [ ]:
# POWODOJE ASSERT! (w can not be a counterexample!)
# seeds = [i+ 1 for i in range(10)]

# input_signs = ['a', 'b', 'c']
# max_number_of_states = 150
# number_of_itreations = len(seeds)

# results = []

# i = 0 
# while i < number_of_itreations: 
#     print(f"iter = {i+1}")
#     random.seed(seeds[i])
#     i += 1

#     reset_word = DFA.NOT_RESETING_WORD
#     trial_nr = 1 
#     min_found_reset_word = 'a' * 10000 
#     while True:
#         dfa_ = DFA()
#         dfa_.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
#         d, cnt_normal, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa_)))   

#         reset_word = d.find_minim_synch_word()
#         print(f"trial_nr = {trial_nr}")
#         if reset_word==DFA.NOT_RESETING_WORD:
#             continue
#         if len(reset_word) < 20:
#             d.type = DFA.SYNCHRONICITY
#             d.reset_word = reset_word
#             break
#         else:
#             trial_nr += 1
#             min_found_reset_word = reset_word if len(reset_word) < len(min_found_reset_word) else min_found_reset_word
#         if trial_nr >= 15: 
#             d.type = DFA.SYNCHRONICITY
#             d.reset_word = min_found_reset_word
#             break
    
#     print(f"zaczynam nauke, len(reset_word) = {len(d.reset_word)}")
#     d1_with_advice, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=d), advice_system=SRS(), check_consistency=True, debug=True) 
#     d1_with_advice.type = DFA.SYNCHRONICITY

#     results.append(RecordSYNCH(d=d.Q, reset_word=reset_word, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, len(cnt_ex_with_advice))))

In [ ]:
seeds = [i+ 1 for i in range(10)]

input_signs = ['a', 'b', 'c']
max_number_of_states = 70
number_of_itreations = len(seeds)

results = []

i = 0 
while i < number_of_itreations: 
    print(f"iter = {i+1}")
    random.seed(seeds[i])
    i += 1

    reset_word = DFA.NOT_RESETING_WORD
    trial_nr, min_found_reset_word = 1, 'a' * 10000 
    while True:
        dfa_ = DFA()
        dfa_.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
        d, cnt_normal, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa_)))   

        reset_word = d.find_minim_synch_word()
        if reset_word==DFA.NOT_RESETING_WORD:
            continue
        if len(reset_word) < 20:
            d.type = DFA.SYNCHRONICITY
            d.reset_word = reset_word
            break
        else:
            trial_nr += 1
            min_found_reset_word = reset_word if len(reset_word) < len(min_found_reset_word) else min_found_reset_word
        if trial_nr >= 5: 
            d.type = DFA.SYNCHRONICITY
            d.reset_word = min_found_reset_word
            break
    
    d1_with_advice, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=d), advice_system=SRS(), check_consistency=True, debug=False) 
    d1_with_advice.type = DFA.SYNCHRONICITY

    results.append(RecordSYNCH(d=d.Q, reset_word=reset_word, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, len(cnt_ex_with_advice))))

In [131]:
results.sort(key=lambda x: x.d)

In [ ]:
columns = [('', 'DFA'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.d for r in results],
    columns[1]: [r.cnt_normal[0] for r in results],
    columns[2]: [r.cnt_normal[1] for r in results],
    columns[3]: [r.cnt_with_advice[0][0] for r in results],
    columns[4]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[5]: [r.mq_reduction for r in results],
    columns[6]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

L* without advice     L* with advice            \
                  DFA                MQ  EQ             MQ        EQ   
0                  34              1828  15           2003    (7, 7)   
1                  35               745   7           1523    (2, 2)   
2                  37              2362  22           2027  (11, 11)   
3                  38               885   8           1773    (3, 3)   
4                  41              2614  18           4988   (7, 10)   
5                  42              1147   8           2257    (2, 3)   
6                  47              3182  23           4879  (12, 13)   
7                  48              1406  10           2621    (3, 3)   
8                  61              4981  23           3938  (11, 11)   
9                  63              1714   8           3050    (3, 3)   
10                                                                     
11      Max reduction                                                  
12      Min reduction                                                  
13  Average reduction                                                  

   Reduction       
          MQ   EQ  
0        -9%  53%  
1      -104%  71%  
2        14%  50%  
3      -100%  62%  
4       -90%  61%  
5       -96%  75%  
6       -53%  47%  
7       -86%  70%  
8        20%  52%  
9       -77%  62%  
10                 
11       20%  75%  
12     -104%  47%  
13      -58%  60%

### Marked words

In [147]:
class RecordMARKED:
    def __init__(self, d, cnt_normal, cnt_with_advice):
        self.d = d
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
    def print_record(self):
        print(f"|d| = {self.d}, |cnt_normal| = {self.cnt_normal} vs |cnt_with_advice| = {self.cnt_with_advice}") 

In [149]:
seeds = [i+1 for i in range(10)]

input_signs = ['a', 'b', 'c', 'd'] 
max_number_of_states = 500
number_of_itreations = len(seeds)

results = []

i = 0 
while i < number_of_itreations:  
    random.seed(seeds[i])
    i += 1

    dfa_ = DFA()
    dfa_.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    d, cnt_normal, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa_)), advice_system=None, check_consistency=False)   
    d_marked = d.create_marked_words_atomaton()

    # _, cnt_normal, _ = run_learning_process(test=test_example(dfa=d_marked))
    _, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=d_marked), advice_system=SRS(), check_consistency=True) 

    results.append(RecordMARKED(d=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, len(cnt_ex_with_advice))))

In [150]:
results.sort(key=lambda x: x.d)

In [ ]:
columns = [('', 'DFA'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.d for r in results],
    columns[1]: [r.cnt_normal[0] for r in results],
    columns[2]: [r.cnt_normal[1] for r in results],
    columns[3]: [r.cnt_with_advice[0][0] for r in results],
    columns[4]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[5]: [r.mq_reduction for r in results],
    columns[6]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

L* without advice     L* with advice            \
                  DFA                MQ  EQ             MQ        EQ   
0                   1                21   1             33    (1, 0)   
1                 277             19413  17          24052  (21, 21)   
2                 301             29229  19          32493  (22, 22)   
3                 305             19541  15          24614  (19, 19)   
4                 324             19784  16          24174  (19, 19)   
5                 358             30458  22          36428  (23, 23)   
6                 383             39483  24          48209  (28, 28)   
7                 402             23335  14          31160  (19, 19)   
8                 447             59495  37          65652  (39, 39)   
9                 487             37037  21          47988  (18, 18)   
10                                                                     
11      Max reduction                                                  
12      Min reduction                                                  
13  Average reduction                                                  

   Reduction        
          MQ    EQ  
0       -57%    0%  
1       -23%  -23%  
2       -11%  -15%  
3       -25%  -26%  
4       -22%  -18%  
5       -19%   -4%  
6       -22%  -16%  
7       -33%  -35%  
8       -10%   -5%  
9       -29%   14%  
10                  
11      -10%   14%  
12      -57%  -35%  
13      -25%  -12%

### DFA with partial specification

In [19]:
class RecordPARTIAL:
    def __init__(self, d, cnt_normal, cnt_with_advice):
        self.d = d
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
    def print_record(self):
        print(f"|d1| = {self.d}, |d_cnt_normal| = {self.cnt_normal} vs |d_cnt_with_partial_advice| = {self.cnt_with_partial_advice}") 

In [ ]:
seeds = [i+1 for i in range(100)]

input_signs = ['a', 'b', 'c', 'd'] #, 'e', 'f']
max_number_of_states = 1000 
number_of_itreations = len(seeds)

results = []

i = 0 
while i < number_of_itreations:  
    random.seed(seeds[i])
    i += 1
    print(f"iter{i}")

    dfa_ = DFA()
    dfa_.create_random_dfa(Q=random.randint(max_number_of_states//3,max_number_of_states), input_signs=input_signs)
    dfa_partial = copy.deepcopy(dfa_)
    dfa_partial.prune()

    dfa_to_learn = dfa_.create_coppy_with_start_sign()

    d, cnt_normal, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa_to_learn)), advice_system=None, check_consistency=False)   
    _, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=dfa_to_learn), advice_system=dfa_partial, check_consistency=True) 

    results.append(RecordPARTIAL(d=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice)))

In [22]:
results.sort(key = lambda x: x.d)

In [23]:
columns = [('', 'DFA'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.d for r in results],
    columns[1]: [r.cnt_normal[0] for r in results],
    columns[2]: [r.cnt_normal[1] for r in results],
    columns[3]: [r.cnt_with_advice[0][0] for r in results],
    columns[4]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[5]: [r.mq_reduction for r in results],
    columns[6]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

L* without advice      L* with advice              \
                   DFA                MQ   EQ             MQ          EQ   
0                  343             86842   47          63236    (46, 47)   
1                  358             36183   19          25108    (18, 19)   
2                  363            379596  186         282934  (185, 186)   
3                  367            161957   84         119493    (83, 84)   
4                  371             89471   40          65043    (39, 40)   
..                 ...               ...  ...            ...         ...   
99                 985            103451   18          71955    (17, 18)   
100                                                                        
101      Max reduction                                                     
102      Min reduction                                                     
103  Average reduction                                                     

    Reduction      
           MQ  EQ  
0         27%  2%  
1         30%  5%  
2         25%  0%  
3         26%  1%  
4         27%  2%  
..        ...  ..  
99        30%  5%  
100                
101       32%  7%  
102       25%  0%  
103       28%  2%  

[104 rows x 7 columns]

### Uczenie sie automatów z indempotentną literką

In [24]:
class RecordIDEM:
    def __init__(self, d, cnt_normal, cnt_with_advice):
        self.d = d
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
    def print_record(self):
        print(f"|d| = {self.d}, |cnt_normal| = {self.cnt_normal} vs |cnt_with_advice| = {self.cnt_with_advice}") 

In [ ]:
seeds = [i+1 for i in range(10)]

input_signs = ['a', 'b', 'c', 'd']
max_number_of_states =1000 
number_of_itreations = len(seeds)

results = []

i = 0 
while i < number_of_itreations:  
    random.seed(seeds[i])
    i += 1
    print(f"iter: {i}")

    dfa_ = DFA()

    dfa_.create_random_indempotent_automaton(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    d, cnt_normal, _ = run_learning_process(test=test_example(dfa=dfa_))
    d.type = DFA.INDEMPOTENT
    while not d.check_if_idempotent():
        dfa_.create_random_indempotent_automaton(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
        d, cnt_normal, _ = run_learning_process(test=test_example(dfa=dfa_))
        d.type = DFA.INDEMPOTENT

    _, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=d), advice_system=SRS(), check_consistency=True, debug=False) 

    results.append(RecordIDEM(d=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice)))

In [220]:
results.sort(key=lambda x: x.d)

In [221]:
columns = [('', 'DFA'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.d for r in results],
    columns[1]: [r.cnt_normal[0] for r in results],
    columns[2]: [r.cnt_normal[1] for r in results],
    columns[3]: [r.cnt_with_advice[0][0] for r in results],
    columns[4]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[5]: [r.mq_reduction for r in results],
    columns[6]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

L* without advice      L* with advice              \
                  DFA                MQ   EQ             MQ          EQ   
0                 185             12973   18          12443    (13, 18)   
1                 225             15097   16          15246    (14, 18)   
2                 271             16551   13          15871    (11, 16)   
3                 276             18514   14          17734    (13, 17)   
4                 280            181095  122         167458  (118, 125)   
5                 325             31557   25          29036    (22, 29)   
6                 388             27183   15          26009    (13, 17)   
7                 439             29435   14          24969    (12, 16)   
8                 448            157813   77         143376    (72, 85)   
9                 468             39808   16          40937    (19, 25)   
10                                                                        
11      Max reduction                                                     
12      Min reduction                                                     
13  Average reduction                                                     

   Reduction        
          MQ    EQ  
0         4%   27%  
1         0%   12%  
2         4%   15%  
3         4%    7%  
4         7%    3%  
5         7%   12%  
6         4%   13%  
7        15%   14%  
8         9%    6%  
9        -2%  -18%  
10                  
11       15%   27%  
12       -2%  -18%  
13        5%    9%

<!-- ### Uczenie się splotów - **debugowanie**  -->